In [49]:
import paddle
import paddle.nn as nn
from paddle.nn import Conv2D, Linear, Embedding
from paddle import to_tensor
import paddle.nn.functional as F
# print(paddle.__version__)

import os, zipfile
import io, random, json
import numpy as np
import matplotlib.pyplot as plt

#解压并读取数据

In [50]:
import os, zipfile
src_path="data/Rumor_Dataset.zip"
target_path="/home/aistudio/data/Chinese_Rumor_Dataset-master"
if(not os.path.isdir(target_path)):
    z = zipfile.ZipFile(src_path, 'r')
    z.extractall(path=target_path)
    z.close()

In [51]:
import io
import random
import json

rumor_class_dirs = os.listdir(target_path+"/Chinese_Rumor_Dataset-master/CED_Dataset/rumor-repost/")

non_rumor_class_dirs = os.listdir(target_path+"/Chinese_Rumor_Dataset-master/CED_Dataset/non-rumor-repost/")

original_microblog = target_path+"/Chinese_Rumor_Dataset-master/CED_Dataset/original-microblog/"

#rumor标签='0',non_rumor标签='1'
rumor_label="0"
non_rumor_label="1"

#统计谣言数据与非谣言数据的总数
rumor_num = 0
non_rumor_num = 0

all_rumor_list = []
all_non_rumor_list = []

#样本信息统计
for rumor_class_dir in rumor_class_dirs: 
    if(rumor_class_dir != '.DS_Store'):
        #遍历谣言数据，并解析
        with open(original_microblog + rumor_class_dir, 'r') as f:
	        rumor_content = f.read()
        rumor_dict = json.loads(rumor_content)
        all_rumor_list.append(rumor_label+"\t"+rumor_dict["text"]+"\n")
        rumor_num +=1

for non_rumor_class_dir in non_rumor_class_dirs: 
    if(non_rumor_class_dir != '.DS_Store'):
        with open(original_microblog + non_rumor_class_dir, 'r') as f2:
	        non_rumor_content = f2.read()
        non_rumor_dict = json.loads(non_rumor_content)
        all_non_rumor_list.append(non_rumor_label+"\t"+non_rumor_dict["text"]+"\n")
        non_rumor_num +=1
        
print("rumor_num="+str(rumor_num))
print("non_rumor_num="+str(non_rumor_num))

rumor_num=1538
non_rumor_num=1849


In [52]:
#全部数据shuffle后写入all_data.txt

data_list_path="/home/aistudio/data/"
all_data_path=data_list_path + "data.txt"

all_data_list = all_rumor_list + all_non_rumor_list

random.shuffle(all_data_list)

#在生成all_data.txt之前，首先将其清空
with open(all_data_path, 'w') as f:
    f.seek(0)
    f.truncate() 
    
with open(all_data_path, 'a') as f:
    for data in all_data_list:
        f.write(data) 

In [53]:
# 生成数据字典
def create_dict(data_path, dict_path):
    with open(dict_path, 'w') as f:
        f.seek(0)
        f.truncate() 

    dict_set = set()
    # 读取全部数据
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # 把数据生成一个元组
    for line in lines:
        content = line.split('\t')[-1].replace('\n', '')
        for s in content:
            dict_set.add(s)
    # 把元组转换成字典，一个字对应一个数字
    dict_list = []
    i = 0
    for s in dict_set:
        dict_list.append([s, i])
        i += 1
    # 添加未知字符
    dict_txt = dict(dict_list)
    end_dict = {"<unk>": i}
    dict_txt.update(end_dict)
    end_dict = {"<pad>": i+1}
    dict_txt.update(end_dict)
    # 把这些字典保存到本地中
    with open(dict_path, 'w', encoding='utf-8') as f:
        f.write(str(dict_txt))

        
    print("Done!")

In [54]:
# 创建序列化表示的数据,并按照一定比例划分数据集为训练数据train.txt与验证数据val.txt
def create_data_list(data_list_path):
    #在生成数据之前，首先将val.txt和train.txt清空
    with open(os.path.join(data_list_path, 'val.txt'), 'w', encoding='utf-8') as f_eval:
        f_eval.seek(0)
        f_eval.truncate()
        
    with open(os.path.join(data_list_path, 'train.txt'), 'w', encoding='utf-8') as f_train:
        f_train.seek(0)
        f_train.truncate() 
    
    with open(os.path.join(data_list_path, 'dict.txt'), 'r', encoding='utf-8') as f_data:
        dict_txt = eval(f_data.readlines()[0])

    with open(os.path.join(data_list_path, 'data.txt'), 'r', encoding='utf-8') as f_data:
        lines = f_data.readlines()
    
    i = 0
    maxlen = 0
    with open(os.path.join(data_list_path, 'val.txt'), 'a', encoding='utf-8') as f_eval,open(os.path.join(data_list_path, 'train.txt'), 'a', encoding='utf-8') as f_train:
        for line in lines:
            words = line.split('\t')[-1].replace('\n', '')
            maxlen = max(maxlen, len(words))
            label = line.split('\t')[0]
            labs = ""
            # 每8个抽取一个数据用于验证
            if i % 5 == 0:
                for s in words:
                    lab = str(dict_txt[s])
                    labs = labs + lab + ','
                labs = labs[:-1]
                labs = labs + '\t' + label + '\n'
                f_eval.write(labs)
            else:
                for s in words:
                    lab = str(dict_txt[s])
                    labs = labs + lab + ','
                labs = labs[:-1]
                labs = labs + '\t' + label + '\n'
                f_train.write(labs)
            i += 1
        
    print("数据列表生成完成！")
    print(maxlen)

In [55]:
# 把生成的数据列表都放在自己的总类别文件夹中
data_root_path = "/home/aistudio/data/" 
data_path = os.path.join(data_root_path, 'data.txt')
dict_path = os.path.join(data_root_path, "dict.txt")

# 创建数据字典
create_dict(data_path, dict_path)

# 创建数据列表
create_data_list(data_root_path)

Done!
数据列表生成完成！
226


In [56]:
# 打印前2条训练数据
vocab = load_vocab(os.path.join(data_root_path, 'dict.txt'))

def ids_to_str(ids):
    words = []
    for k in ids:
        w = list(vocab.keys())[list(vocab.values()).index(int(k))]
        words.append(w if isinstance(w, str) else w.decode('ASCII'))
    return " ".join(words)

file_path = os.path.join(data_root_path, 'train.txt')
with io.open(file_path, "r", encoding='utf8') as fin:
        i = 0
        for line in fin:
            i += 1
            cols = line.strip().split("\t")
            if len(cols) != 2:
                sys.stderr.write("[NOTICE] Error Format Line!")
                continue
            label = int(cols[1])
            wids = cols[0].split(",")
            print(str(i)+":")
            print('sentence list id is:', wids)
            print('sentence list is: ', ids_to_str(wids))
            print('sentence label id is:', label)
            print('---------------------------------')
            
            if i == 2: break

1:
sentence list id is: ['2608', '1939', '4288', '707', '4296', '3433', '2608', '1192', '1186', '959', '2156', '4388', '2773', '2490', '1152', '3152', '731', '1009', '2816', '1036', '1476', '2836', '4262', '640', '3221', '1085', '3378', '56', '298', '3945', '4031', '1733', '2309', '4188', '488', '1852', '4313', '1351', '577', '4262', '675', '210', '2773', '1152', '24', '2156', '1553', '1361', '1042', '396', '1733', '2638', '1941', '24', '1351', '2156', '406', '1602', '1234', '121', '769', '2656', '3380', '1085', '3378', '56', '3825', '2838', '3880', '265', '2156', '788', '2755', '676', '3583', '3982', '1245', '915', '1085', '436', '406', '488', '1676', '298', '675', '210', '3825', '2341', '4198', '2448', '1671', '2374', '2448', '3381', '2633', '2448', '3346', '1226', '2448', '1338', '254', '2037', '2448', '1981', '2577', '2843', '2448', '1952', '2770', '2194', '2448', '3191', '863', '2448', '361', '2712', '2448', '1906', '2374', '2448', '3381', '1069', '1085', '1852', '1152', '3581', '

In [57]:
def load_vocab(file_path):
    fr = open(file_path, 'r', encoding='utf8')
    vocab = eval(fr.read())   #读取的str转换为字典
    fr.close()

    return vocab

In [58]:
vocab = load_vocab(os.path.join(data_root_path, 'dict.txt'))

class RumorDataset(paddle.io.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.all_data = []
       
        with io.open(self.data_dir, "r", encoding='utf8') as fin:
            for line in fin:
                cols = line.strip().split("\t")
                if len(cols) != 2:
                    sys.stderr.write("[NOTICE] Error Format Line!")
                    continue
                label = []
                label.append(int(cols[1]))
                wids = cols[0].split(",")
                if len(wids)>=150:
                    wids = np.array(wids[:150]).astype('int64')     
                else:
                    wids = np.concatenate([wids, [vocab["<pad>"]]*(150-len(wids))]).astype('int64')
                label = np.array(label).astype('int64')
                self.all_data.append((wids, label))

        
    def __getitem__(self, index):
        data, label = self.all_data[index]
        return data, label

    def __len__(self):
        return len(self.all_data)


batch_size = 32
train_dataset = RumorDataset(os.path.join(data_root_path, 'train.txt'))
test_dataset = RumorDataset(os.path.join(data_root_path, 'val.txt'))

train_loader = paddle.io.DataLoader(train_dataset, places=paddle.CPUPlace(), return_list=True,
                                    shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = paddle.io.DataLoader(test_dataset, places=paddle.CPUPlace(), return_list=True,
                                    shuffle=True, batch_size=batch_size, drop_last=True)




#检查

print('train_dataset') 
for data, label in train_dataset:
    print(data)
    print(np.array(data).shape)
    print(label)
    break


print('test_dataset') 
for data, label in test_dataset:
    print(data)
    print(np.array(data).shape)
    print(label)
    break

train_dataset
[2608 1939 4288  707 4296 3433 2608 1192 1186  959 2156 4388 2773 2490
 1152 3152  731 1009 2816 1036 1476 2836 4262  640 3221 1085 3378   56
  298 3945 4031 1733 2309 4188  488 1852 4313 1351  577 4262  675  210
 2773 1152   24 2156 1553 1361 1042  396 1733 2638 1941   24 1351 2156
  406 1602 1234  121  769 2656 3380 1085 3378   56 3825 2838 3880  265
 2156  788 2755  676 3583 3982 1245  915 1085  436  406  488 1676  298
  675  210 3825 2341 4198 2448 1671 2374 2448 3381 2633 2448 3346 1226
 2448 1338  254 2037 2448 1981 2577 2843 2448 1952 2770 2194 2448 3191
  863 2448  361 2712 2448 1906 2374 2448 3381 1069 1085 1852 1152 3581
 1553  396 3306  675 3385 1085 4410 4410 4410 4410 4410 4410 4410 4410
 4410 4410 4410 4410 4410 4410 4410 4410 4410 4410]
(150,)
[1]
test_dataset
[ 672 2288 1086 3120 2579  974 2410  519 1838 2263 3658 3982 1263 1568
  671 3233 2504 3457  352 1733 3805  939 3805 1841 1045  284 3615 4188
 3359 4091  820 3026 4304  629 3825 3745 3946 3889   94 38

In [59]:
## 导入模型
from model_utils import Transformer,CNN

In [60]:
#定义卷积网络
class Conv(nn.Layer):
    def __init__(self,dict_dim):
        super(Conv,self).__init__()
        self.dict_dim = dict_dim
        self.emb_dim = 128
        self.hid_dim = 128
        self.fc_hid_dim = 96
        self.class_dim = 2
        self.channels = 1
        self.win_size = [3, self.hid_dim]
        self.batch_size = 32
        self.seq_len = 150
        self.embedding = Embedding(self.dict_dim + 1, self.emb_dim, sparse=False)
        self.hidden1 = paddle.nn.Conv2D(in_channels=1,                        #通道数
                                            out_channels=self.hid_dim,        #卷积核个数
                                            kernel_size=self.win_size,        #卷积核大小
                                            padding=[1, 1]
                                            )                         
        self.relu1 = paddle.nn.ReLU()
        self.hidden3 = paddle.nn.MaxPool2D(kernel_size=2,         #池化核大小
                                            stride=2)             #池化步长2
        self.hidden4 = paddle.nn.Linear(128*75, 2)
    #网络的前向计算过程
    def forward(self,input):
        
        #print('输入维度：', input.shape)
        x = self.embedding(input)
        x = paddle.reshape(x, [32, 1, 150, 128])   
        x = self.hidden1(x)
        x = self.relu1(x)
        #print('第一层卷积输出维度：', x.shape)
        x = self.hidden3(x)
        #print('池化后输出维度：', x.shape)
        #在输入全连接层时，需将特征图拉平会自动将数据拉平.

        x = paddle.reshape(x, shape=[self.batch_size, -1])
        out = self.hidden4(x)
        return out

In [61]:
# vocab_size = len(vocab)  
# maxlen = 200  
# seq_len = 200
# batch_size = 32
# epochs = 100
# pad_id = vocab['<pad>']
# embed_dim = 128  # Embedding size for each token
# num_heads = 2  # Number of attention heads
# feed_dim = 128  # Hidden layer size in feed forward network inside transformer
# classes = ['0', '1']

# transformer_model = Transformer(maxlen,vocab_size,embed_dim,num_heads,feed_dim)
# paddle.summary(transformer_model,(200,128),"int64")

Transformer模型

In [62]:
vocab_size = len(vocab)  
maxlen = 200  
seq_len = 200
batch_size = 32
epochs = 100
pad_id = vocab['<pad>']
embed_dim = 128  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_dim = 128  # Hidden layer size in feed forward network inside transformer
classes = ['0', '1']

transformer_model = Transformer(maxlen,vocab_size,embed_dim,num_heads,feed_dim)
paddle.summary(transformer_model,(200,128),"int64")

-----------------------------------------------------------------------------------------
        Layer (type)              Input Shape          Output Shape         Param #    
         Embedding-8                [[128]]             [128, 128]          25,600     
         Embedding-7              [[200, 128]]       [200, 128, 128]        564,608    
 TokenAndPositionEmbedding-3      [[200, 128]]       [200, 128, 128]           0       
          Linear-19            [[200, 128, 128]]     [200, 128, 128]        16,512     
          Linear-20            [[200, 128, 128]]     [200, 128, 128]        16,512     
          Linear-21            [[200, 128, 128]]     [200, 128, 128]        16,512     
          Linear-22            [[200, 128, 128]]     [200, 128, 128]        16,512     
  MultiHeadSelfAttention-3     [[200, 128, 128]]     [200, 128, 128]           0       
          Dropout-9            [[200, 128, 128]]     [200, 128, 128]           0       
         LayerNorm-5          

{'total_params': 692414, 'trainable_params': 692414}

模型训练

In [63]:
def train(model, num_epochs, model_name):
    model.train()
    opt = paddle.optimizer.Adam(learning_rate=0.0005, parameters=model.parameters())
    
    steps = 0
    Iters, total_loss, total_acc = [], [], []
    
    for epoch in range(num_epochs):
        for batch_id, data in enumerate(train_loader):
            steps += 1
            sent = data[0]
            label = data[1]
            
            logits = model(sent)
            loss = paddle.nn.functional.cross_entropy(logits, label)
            acc = paddle.metric.accuracy(logits, label)

            if batch_id % 50 == 0:
                Iters.append(steps)
                total_loss.append(loss.numpy())
                total_acc.append(acc.numpy())

                print("epoch: {}, batch_id: {}, loss is: {}".format(epoch, batch_id, loss.numpy()))
            
            loss.backward()
            opt.step()
            opt.clear_grad()

        # evaluate model after one epoch
        model.eval()
        accuracies = []
        losses = []
        
        for batch_id, data in enumerate(test_loader):
            
            sent = data[0]
            label = data[1]

            logits = model(sent)
            loss = paddle.nn.functional.cross_entropy(logits, label)
            acc = paddle.metric.accuracy(logits, label)
            
            accuracies.append(acc.numpy())
            losses.append(loss.numpy())
        
        avg_acc, avg_loss = np.mean(accuracies), np.mean(losses)

        print("[validation] accuracy: {}, loss: {}".format(avg_acc, avg_loss))
        
        model.train()

    paddle.save(model.state_dict(), model_name + "_model_final.pdparams")
    
    # draw_process("trainning loss","red",Iters,total_loss,"trainning loss")
    # draw_process("trainning acc","green",Iters,total_acc,"trainning acc")

生成模型

In [64]:
print(transformer_model)

Transformer(
  (emb): TokenAndPositionEmbedding(
    (token_emb): Embedding(4411, 128, sparse=False)
    (pos_emb): Embedding(200, 128, sparse=False)
  )
  (trs): TransformerBlock(
    (att): MultiHeadSelfAttention(
      (query_dense): Linear(in_features=128, out_features=128, dtype=float32)
      (key_dense): Linear(in_features=128, out_features=128, dtype=float32)
      (value_dense): Linear(in_features=128, out_features=128, dtype=float32)
      (combine_heads): Linear(in_features=128, out_features=128, dtype=float32)
    )
    (ffn): PointWiseFeedForwardNetwork(
      (linear1): Linear(in_features=128, out_features=128, dtype=float32)
      (softmax): Softmax(axis=-1)
      (linear2): Linear(in_features=128, out_features=128, dtype=float32)
    )
    (layernorm1): LayerNorm(normalized_shape=[128], epsilon=1e-06)
    (layernorm2): LayerNorm(normalized_shape=[128], epsilon=1e-06)
    (dropout1): Dropout(p=0.1, axis=None, mode=upscale_in_train)
    (dropout2): Dropout(p=0.1, axis=Non

In [65]:
import time
start_time=time.time()
train(transformer_model, epochs, "transformer")
end_time=time.time()
running_time=end_time-start_time
print("running_time is {}".format(running_time))

epoch: 0, batch_id: 0, loss is: 0.7064839601516724
epoch: 0, batch_id: 50, loss is: 0.624210000038147
[validation] accuracy: 0.7901785969734192, loss: 0.5301868915557861
epoch: 1, batch_id: 0, loss is: 0.5464376211166382
epoch: 1, batch_id: 50, loss is: 0.4608421325683594
[validation] accuracy: 0.824404776096344, loss: 0.47930464148521423
epoch: 2, batch_id: 0, loss is: 0.4322771430015564
epoch: 2, batch_id: 50, loss is: 0.40623539686203003
[validation] accuracy: 0.84375, loss: 0.4638700485229492
epoch: 3, batch_id: 0, loss is: 0.35959360003471375
epoch: 3, batch_id: 50, loss is: 0.3776213526725769
[validation] accuracy: 0.8586309552192688, loss: 0.44555211067199707
epoch: 4, batch_id: 0, loss is: 0.34895628690719604
epoch: 4, batch_id: 50, loss is: 0.3528658151626587
[validation] accuracy: 0.84375, loss: 0.45778506994247437
epoch: 5, batch_id: 0, loss is: 0.353611022233963
epoch: 5, batch_id: 50, loss is: 0.4396049976348877
[validation] accuracy: 0.8645833134651184, loss: 0.4472208619

In [66]:
model_state_dict = paddle.load('transformer_model_final.pdparams')
transformer_model.set_state_dict(model_state_dict) 
transformer_model.eval()
accuracies = []
losses = []

for batch_id, data in enumerate(test_loader):
    
    sent = data[0]
    label = data[1]

    logits = transformer_model(sent)
    loss = paddle.nn.functional.cross_entropy(logits, label)
    acc = paddle.metric.accuracy(logits, label)
    
    accuracies.append(acc.numpy())
    losses.append(loss.numpy())

avg_acc, avg_loss = np.mean(accuracies), np.mean(losses)
print("[validation] accuracy: {}, loss: {}".format(avg_acc, avg_loss))

[validation] accuracy: 0.8645833134651184, loss: 0.44808441400527954


In [67]:
import numpy as np
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
model_state_dict = paddle.load('model_final.pdparams')
transformer_model.set_state_dict(model_state_dict) 
transformer_model.eval()
predictions = []
r = []
for batch_id, data in enumerate(test_loader):
    sent = data[0]
    gt_labels = data[1].numpy()
    for i in gt_labels:
        r.append(i)
    results = transformer_model(sent)
    for probs in results:
        # 映射分类label
        idx = np.argmax(probs)
        predictions.append(idx)
    
confusion_matrix(r, predictions)
from sklearn.metrics import classification_report
target_names = ["0","1"]
acc = accuracy_score(r, predictions).round(4)
pre = precision_score(r, predictions).round(4)
rec = recall_score(r, predictions).round(4)
F1 = f1_score(r, predictions).round(4)
print('acc:{},pre:{},rec:{},f1:{}'.format(acc,pre,rec,F1))
CR=classification_report(r, predictions, target_names=target_names,digits=4)
print(CR)

acc:0.8616,pre:0.8646,rec:0.8671,f1:0.8658
              precision    recall  f1-score   support

           0     0.8585    0.8558    0.8571       326
           1     0.8646    0.8671    0.8658       346

    accuracy                         0.8616       672
   macro avg     0.8615    0.8614    0.8615       672
weighted avg     0.8616    0.8616    0.8616       672



/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for emb.token_emb.weight. emb.token_emb.weight is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for emb.pos_emb.weight. emb.pos_emb.weight is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for trs.att.query_dense.weight. trs.att.query_dense.weight is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for trs.att.query_dense.bias. trs.att.query_dense.bias is not found in the

In [68]:

label_map = {0:"谣言", 1:"不是谣言"}

model_state_dict = paddle.load('model_final.pdparams')
transformer_model.set_state_dict(model_state_dict) 
transformer_model.eval()

for batch_id, data in enumerate(test_loader):
    
    sent = data[0]
    gt_labels = data[1].numpy()
   
    results = transformer_model(sent)

    predictions = []
    for probs in results:
        # 映射分类label
        idx = np.argmax(probs)
        labels = label_map[idx]
        predictions.append(labels)
    
    for i,pre in enumerate(predictions):
        print('数据: {} \n\n预测: {} \n原始标签：{}'.format(ids_to_str(sent[0]).replace(" ", "").replace("<pad>",""), pre, label_map[gt_labels[0][0]]))
        break
    break

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for emb.token_emb.weight. emb.token_emb.weight is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for emb.pos_emb.weight. emb.pos_emb.weight is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for trs.att.query_dense.weight. trs.att.query_dense.weight is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for trs.att.query_dense.bias. trs.att.query_dense.bias is not found in the

数据: #熊先生一周岁#我的熊宝宝在大家的关爱下已经马上要一岁了，这一年里面是大家一把屎一把尿把我的熊宝宝喂养大的，我们也会在熊先生的生日月带来各种活动，有跟跨界品牌之间的互动合作，有跟艺人之间的定制款，有跟潮流牌品的限定定制款，还有一系列回馈老顾主的活动，感谢熊粉！熊粉万岁！@熊先生品牌 

预测: 不是谣言 
原始标签：不是谣言


CNN模型

In [69]:
cnn_model = Conv(dict_dim=vocab["<pad>"])

import time
start_time=time.time()
train(cnn_model, epochs, "cnn")
end_time=time.time()
running_time=end_time-start_time
print("running_time is {}".format(running_time))

model_state_dict = paddle.load('cnn_model_final.pdparams')
cnn_model.set_state_dict(model_state_dict) 
cnn_model.eval()
accuracies = []
losses = []

for batch_id, data in enumerate(test_loader):
    
    sent = data[0]
    label = data[1]

    logits = cnn_model(sent)
    loss = paddle.nn.functional.cross_entropy(logits, label)
    acc = paddle.metric.accuracy(logits, label)
    
    accuracies.append(acc.numpy())
    losses.append(loss.numpy())

avg_acc, avg_loss = np.mean(accuracies), np.mean(losses)
print("[validation] accuracy: {}, loss: {}".format(avg_acc, avg_loss))

import numpy as np
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
model_state_dict = paddle.load('model_final.pdparams')
transformer_model.set_state_dict(model_state_dict) 
transformer_model.eval()
predictions = []
r = []
for batch_id, data in enumerate(test_loader):
    
    sent = data[0]
    gt_labels = data[1].numpy()
    for i in gt_labels:
        r.append(i)
    results = transformer_model(sent)
    for probs in results:
        # 映射分类label
        idx = np.argmax(probs)
        predictions.append(idx)
    
confusion_matrix(r, predictions)
from sklearn.metrics import classification_report
target_names = ["0","1"]
acc = accuracy_score(r, predictions).round(4)
pre = precision_score(r, predictions).round(4)
rec = recall_score(r, predictions).round(4)
F1 = f1_score(r, predictions).round(4)
print('acc:{},pre:{},rec:{},f1:{}'.format(acc,pre,rec,F1))
CR=classification_report(r, predictions, target_names=target_names,digits=4)
print(CR)


epoch: 0, batch_id: 0, loss is: 0.6830382347106934
epoch: 0, batch_id: 50, loss is: 0.6812448501586914
[validation] accuracy: 0.5565476417541504, loss: 0.6595727801322937
epoch: 1, batch_id: 0, loss is: 0.6040115356445312
epoch: 1, batch_id: 50, loss is: 0.4054224491119385
[validation] accuracy: 0.8407738208770752, loss: 0.39349228143692017
epoch: 2, batch_id: 0, loss is: 0.13129544258117676
epoch: 2, batch_id: 50, loss is: 0.2221955955028534
[validation] accuracy: 0.8407738208770752, loss: 0.36038738489151
epoch: 3, batch_id: 0, loss is: 0.11491723358631134
epoch: 3, batch_id: 50, loss is: 0.14197418093681335
[validation] accuracy: 0.8616071343421936, loss: 0.35448211431503296
epoch: 4, batch_id: 0, loss is: 0.08386333286762238
epoch: 4, batch_id: 50, loss is: 0.02135905623435974
[validation] accuracy: 0.855654776096344, loss: 0.4239746332168579
epoch: 5, batch_id: 0, loss is: 0.03588009625673294
epoch: 5, batch_id: 50, loss is: 0.025930725038051605
[validation] accuracy: 0.8601190447